# APS360 Final Project: Covid Mask Detector

### Names: Edwin Zhang, Lance Canlas, Stefano Katsoras, Marie Joy Cuevas

##### Description/Info/whaterver: 
placeholder

## Global Imports

In [7]:
import matplotlib.pyplot as plt

## Loading Our Data

In [5]:
import os



str